In [ ]:
import pandas as pd
import json
from datetime import datetime
import os
import glob
import jsongit 

In [38]:

with open('../data/raw_openweather_20250820_000410.json', 'r') as f:
    ow_data = json.load(f)


In [42]:
records = []
for city, info in ow_data.items():
    
    record = {
        "city": info.get("name", city),
        "country": info.get("sys", {}).get("country"),
        "lat": info.get("coord", {}).get("lat"),
        "lon": info.get("coord", {}).get("lon"),
        "temp": info.get("main", {}).get("temp"),
        "feels_like": info.get("main", {}).get("feels_like"),
        "temp_min": info.get("main", {}).get("temp_min"),
        "temp_max": info.get("main", {}).get("temp_max"),
        "pressure": info.get("main", {}).get("pressure"),
        "humidity": info.get("main", {}).get("humidity"),
        "wind_speed": info.get("wind", {}).get("speed"),
        "wind_deg": info.get("wind", {}).get("deg"),
        "weather_main": info.get("weather", [{}])[0].get("main"),
        "weather_desc": info.get("weather", [{}])[0].get("description"),
    }
    records.append(record)

# Criar DataFrame
df_weather = pd.DataFrame(records)

# Renomear colunas para português
df_weather = df_weather.rename(columns={
    "city": "cidade",
    "country": "país",
    "lat": "latitude",
    "lon": "longitude",
    "temp": "temperatura",
    "feels_like": "sensação_térmica",
    "temp_min": "temperatura_mínima",
    "temp_max": "temperatura_máxima",
    "pressure": "pressão",
    "humidity": "umidade",
    "wind_speed": "vento_velocidade",
    "wind_deg": "vento_direção",
    "weather_main": "clima",
    "weather_desc": "descrição_clima"
})

df_weather


,cidade,país,latitude,longitude,temperatura,sensação_térmica,temperatura_mínima,temperatura_máxima,pressão,umidade,vento_velocidade,vento_direção,clima,descrição_clima
0,São Paulo,BR,-23.5475,-46.6361,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
1,Rio de Janeiro,BR,-22.9028,-43.2075,294.13,294.06,292.15,294.13,1019,68,3.60,110,Clear,clear sky
2,Belo Horizonte,BR,-19.9208,-43.9378,288.36,287.74,288.36,288.36,1023,69,2.35,93,Clouds,broken clouds


In [ ]:

with open("../data/raw_openaq_20250820_000410.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [40]:


stations_list = []

# Poluentes que queremos como colunas
pollutants = ["co", "no2", "o3", "pm10", "pm25", "so2"]

for city, stations in data.items():
    for station in stations:
        row = {
            "id": station["id"],
            "name": station["name"],
            "city": city,
            "latitude": station["coordinates"]["latitude"],
            "longitude": station["coordinates"]["longitude"]
        }
        
        # Inicializar colunas de poluentes como 0
        for pol in pollutants:
            row[pol] = 0
        
        # Preencher poluentes com 1 se estiverem presentes nos sensores
        for sensor in station.get("sensors", []):
            param_dict = sensor.get("parameter", {})
            param_name = param_dict.get("name")
            if param_name in pollutants:
                row[param_name] = 1
        
        stations_list.append(row)

# Criar DataFrame
df_stations = pd.DataFrame(stations_list)
df_stations = df_stations[["id", "name", "city", "latitude", "longitude"] + pollutants]

df_stations


,id,name,city,latitude,longitude,co,no2,o3,pm10,pm25,so2
0,5233,Cid.Universitária-USP-Ipen,São Paulo,-23.566342,-46.737414,0,1,1,0,1,0
1,5243,Pinheiros,São Paulo,-23.561460,-46.702017,1,1,1,1,1,0
2,5252,Congonhas,São Paulo,-23.616320,-46.663466,1,1,0,1,1,1
3,5258,Itaquera,São Paulo,-23.580015,-46.466651,0,0,1,0,0,0
4,5268,Capão Redondo,São Paulo,-23.668356,-46.780043,0,0,1,0,0,0
5,5277,Ibirapuera,São Paulo,-23.591842,-46.660688,1,1,1,0,1,0
6,5278,Interlagos,São Paulo,-23.680508,-46.675043,0,1,1,1,0,1
7,5279,Itaim Paulista,São Paulo,-23.501547,-46.420737,0,1,1,1,1,0
8,5282,Parque D.Pedro II,São Paulo,-23.544846,-46.627676,1,1,1,1,1,0
9,5284,Marg.Tietê-Pte Remédios,São Paulo,-23.518706,-46.743320,1,1,0,0,1,1


In [44]:
df_merged = pd.merge(
    df_stations,
    df_weather,
    left_on="city",    # coluna do df de estações
    right_on="cidade", # coluna do df de clima
    how="left"         # manter todas as estações, mesmo que não tenha clima
)

# Opcional: remover a coluna duplicada "cidade" após o merge
df_merged = df_merged.drop(columns=["cidade"])

# Visualizar o resultado
df_merged

,id,name,city,latitude_x,longitude_x,co,no2,o3,pm10,pm25,...,temperatura,sensação_térmica,temperatura_mínima,temperatura_máxima,pressão,umidade,vento_velocidade,vento_direção,clima,descrição_clima
0,5233,Cid.Universitária-USP-Ipen,São Paulo,-23.566342,-46.737414,0,1,1,0,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
1,5243,Pinheiros,São Paulo,-23.561460,-46.702017,1,1,1,1,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
2,5252,Congonhas,São Paulo,-23.616320,-46.663466,1,1,0,1,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
3,5258,Itaquera,São Paulo,-23.580015,-46.466651,0,0,1,0,0,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
4,5268,Capão Redondo,São Paulo,-23.668356,-46.780043,0,0,1,0,0,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
5,5277,Ibirapuera,São Paulo,-23.591842,-46.660688,1,1,1,0,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
6,5278,Interlagos,São Paulo,-23.680508,-46.675043,0,1,1,1,0,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
7,5279,Itaim Paulista,São Paulo,-23.501547,-46.420737,0,1,1,1,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
8,5282,Parque D.Pedro II,São Paulo,-23.544846,-46.627676,1,1,1,1,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
9,5284,Marg.Tietê-Pte Remédios,São Paulo,-23.518706,-46.743320,1,1,0,0,1,...,290.42,290.29,290.42,291.35,1020,80,4.12,60,Clear,clear sky
